## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Возможные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/data/women-clothing-accessories.csv", sep="\t", usecols=[0, 1])

In [ ]:
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
from sklearn.metrics import *
import nltk
from nltk.tokenize import word_tokenize
from nltk import tokenize
from nltk.corpus import stopwords
import string

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 822.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d80b1884cc5711993f2119f794d85fb6c7733bb7e7425bad2a915f518c01c663
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install pymystem3
from pymystem3 import Mystem

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

def tokenization(text):
  return word_tokenize(text)

def to_lower_case(tokens):
  lower_case_tokens = []

  for token in tokens:
    lower_case_tokens.append(token.lower())

  return lower_case_tokens

def remove_stop_words(tokens):
  stop_words = stopwords.words('russian')

  filtered_tokens = []

  for token in tokens:
    if token not in stop_words:
      filtered_tokens.append(token)

  return filtered_tokens


def preprocessing(text, _tokenization=False, _to_lower_case=False, _remove_stop_words=False):
  if _tokenization:
    tokens = tokenization(text)
  else:
    tokens = text.split()

  if _to_lower_case:
    tokens = to_lower_case(tokens)

  if _remove_stop_words:
    tokens = remove_stop_words(tokens)

  return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.metrics import classification_report

def classification_report_custom(vectorizer, X, y):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
   vectorized_x_train = vectorizer.fit_transform(X_train)
   vectorized_x_test = vectorizer.transform(X_test)

   classifier = MultinomialNB()
   classifier.fit(vectorized_x_train, y_train)

   pred = classifier.predict(vectorized_x_test)
   precision = precision_score(y_test, pred, average='weighted')
   recall = recall_score(y_test, pred, average='weighted')
   f1 = f1_score(y_test, pred, average='weighted')
   accuracy = accuracy_score(y_test, pred)

   return {'Precision': precision, 'Recall': recall, 'F1-Score': f1, 'Accuracy': accuracy}

def generate_vectorizer_data(X, y):
  results = []

  ngram_ranges = [(1, 3), (1, 4), (2, 3)]
  ngram_ranges_Tfidf = [(1, 3), (1, 4), (2, 3)]
  max_df_values = [0.2, 0.3]
  min_df_values = [0.01, 0.02]
  max_features_values = [2500, 10000, 30000]
  char_ngram_ranges = [(3, 5), (4, 6), (5, 7)]

  for ngram_range in ngram_ranges:
    vectorizer_config = {'Type': f'CountVectorizer ngram_range {ngram_range}'}
    metrics = classification_report_custom(CountVectorizer(ngram_range=ngram_range), X, y)
    results.append({**vectorizer_config, **metrics})

  for ngram_range_Tfidf in ngram_ranges_Tfidf:
    for max_df in max_df_values:
      for min_df in min_df_values:
        for max_features in max_features_values:
           vectorizer_config = {'Type': f'TfidfVectorizer ngram_range{ngram_range_Tfidf}, max_df {max_df}, min_df {min_df}, max_features {max_features}'}
           metrics = classification_report_custom(TfidfVectorizer(ngram_range=ngram_range_Tfidf, max_df=max_df, min_df=min_df, max_features=max_features), X, y)
           results.append({**vectorizer_config, **metrics})


  for char_ngram_range in char_ngram_ranges:
    vectorizer_config = {'Type': f'CountVectorizer char_ngram_range{char_ngram_range}'}
    metrics = classification_report_custom(CountVectorizer(analyzer='char', ngram_range=char_ngram_range), X, y)
    results.append({**vectorizer_config, **metrics})

  return results


In [ ]:
preprocessing_options = [(_tokenization, _to_lower_case, _remove_stop_words)
                         for _tokenization in [False, True]
                         for _to_lower_case in [False, True]
                         for _remove_stop_words in [False, True]]

all_results = []

for options in preprocessing_options:
  x = data.review.apply(lambda text: preprocessing(text, *options))
  y = data.sentiment

  results = generate_vectorizer_data(x, y)
  for result in results:
    result.update(zip(['Tokenization', 'To Lower Case', 'Remove Stop Words'], options))

  all_results.extend(results)

df = pd.DataFrame(all_results)
df

,Type,Precision,Recall,F1-Score,Accuracy,Tokenization,To Lower Case,Remove Stop Words
0,"CountVectorizer ngram_range (1, 3)",0.733884,0.732054,0.732805,0.732054,False,False,False
1,"CountVectorizer ngram_range (1, 4)",0.732365,0.730909,0.731571,0.730909,False,False,False
2,"CountVectorizer ngram_range (2, 3)",0.706657,0.707003,0.706651,0.707003,False,False,False
3,"TfidfVectorizer ngram_range(1, 3), max_df 0.2,...",0.663354,0.660976,0.661498,0.660976,False,False,False
4,"TfidfVectorizer ngram_range(1, 3), max_df 0.2,...",0.663354,0.660976,0.661498,0.660976,False,False,False
...,...,...,...,...,...,...,...,...
331,"TfidfVectorizer ngram_range(2, 3), max_df 0.3,...",0.555848,0.363603,0.231246,0.363603,True,True,True
332,"TfidfVectorizer ngram_range(2, 3), max_df 0.3,...",0.555848,0.363603,0.231246,0.363603,True,True,True
333,"CountVectorizer char_ngram_range(3, 5)",0.710178,0.700808,0.702403,0.700808,True,True,True
334,"CountVectorizer char_ngram_range(4, 6)",0.716396,0.709360,0.710575,0.709360,True,True,True


In [ ]:
df.sort_values(by='F1-Score', ascending=False).head(15)

,Type,Precision,Recall,F1-Score,Accuracy,Tokenization,To Lower Case,Remove Stop Words
168,"CountVectorizer ngram_range (1, 3)",0.733889,0.732054,0.732808,0.732054,True,False,False
252,"CountVectorizer ngram_range (1, 3)",0.733889,0.732054,0.732808,0.732054,True,True,False
84,"CountVectorizer ngram_range (1, 3)",0.733884,0.732054,0.732805,0.732054,False,True,False
0,"CountVectorizer ngram_range (1, 3)",0.733884,0.732054,0.732805,0.732054,False,False,False
253,"CountVectorizer ngram_range (1, 4)",0.732370,0.730909,0.731574,0.730909,True,True,False
169,"CountVectorizer ngram_range (1, 4)",0.732370,0.730909,0.731574,0.730909,True,False,False
1,"CountVectorizer ngram_range (1, 4)",0.732365,0.730909,0.731571,0.730909,False,False,False
85,"CountVectorizer ngram_range (1, 4)",0.732365,0.730909,0.731571,0.730909,False,True,False
293,"CountVectorizer char_ngram_range(5, 7)",0.735310,0.728283,0.729835,0.728283,True,True,False
209,"CountVectorizer char_ngram_range(5, 7)",0.735310,0.728283,0.729835,0.728283,True,False,False


## Вывод:
1. Лемматизация не была использована, так как большие объемы данных требуют достаточного количества времени для обработки. С учетом использования самого быстрого лемматизатора от Яндекса(Mystem), выполнение было больше 1 часа.
2. Превысить точность 0.73 не получилось.

Наиболее удачные попытки:
Без препроцессинга
* CountVectorizer(ngram_range=(1, 3))
* CountVectorizer(ngram_range=(1, 4))


Приведение в нижний регистр
* CountVectorizer(ngram_range=(1, 3))
* CountVectorizer(ngram_range=(1, 4))

Токенизация
* CountVectorizer(ngram_range=(1, 3))
* CountVectorizer(ngram_range=(1, 4))
* CountVectorizer(analyzer='char', ngram_range=(5, 7))

Токенизация + приведение в нижний регистр
* CountVectorizer(ngram_range=(1, 3))
* CountVectorizer(ngram_range=(1, 4))
* CountVectorizer(analyzer='char', ngram_range=(5, 7))

Удаление стоп слов не повлияло на улучшение результата





## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [2]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [3]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [4]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [5]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [6]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**abcd efghijk abca bcxabc**
abcx пересекается с abca, а findall возвращает список всех найденных непересекающихся совпадений.

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [7]:
string = "If the discussion veers to the fact that string theory predicts a landscape and hence makes no predictions, some string theorists will rhapsodize about changing the definition of science."
print(re.findall(r'\b\w{2}', string))


['If', 'th', 'di', 've', 'to', 'th', 'fa', 'th', 'st', 'th', 'pr', 'la', 'an', 'he', 'ma', 'no', 'pr', 'so', 'st', 'th', 'wi', 'rh', 'ab', 'ch', 'th', 'de', 'of', 'sc']


### split
разделяет строку по заданному шаблону


In [8]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [9]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [10]:
text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industrys standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum. Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source."
print(re.split('\.', text, maxsplit=3))

['Lorem Ipsum is simply dummy text of the printing and typesetting industry', ' Lorem Ipsum has been the industrys standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book', ' It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged', ' It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum. Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoub

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [11]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [12]:
string = "1027 is a number, but 999 is also a number"

print(re.sub('\d+', 'DIG', string))

DIG is a number, but DIG is also a number


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [13]:
string = "some site https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python and another one https://isu.ifmo.ru/"

print(re.sub(r'http[s]?://[\w.?/=:\-]+', "", string))

some site  and another one 


### compile
компилирует регулярное выражение в отдельный объект

In [14]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [15]:
string = "It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using 'Content here, content here', making it look like readable English. Many desktop publishing packages and web page editors now use Lorem Ipsum as their default model text, and a search for 'lorem ipsum' will uncover many web sites still in their infancy. Various versions have evolved over the years, sometimes by accident, sometimes on purpose injected humour and the like."
prog = re.compile(r'\w{4,}')
prog.findall(string)

['long',
 'established',
 'fact',
 'that',
 'reader',
 'will',
 'distracted',
 'readable',
 'content',
 'page',
 'when',
 'looking',
 'layout',
 'point',
 'using',
 'Lorem',
 'Ipsum',
 'that',
 'more',
 'less',
 'normal',
 'distribution',
 'letters',
 'opposed',
 'using',
 'Content',
 'here',
 'content',
 'here',
 'making',
 'look',
 'like',
 'readable',
 'English',
 'Many',
 'desktop',
 'publishing',
 'packages',
 'page',
 'editors',
 'Lorem',
 'Ipsum',
 'their',
 'default',
 'model',
 'text',
 'search',
 'lorem',
 'ipsum',
 'will',
 'uncover',
 'many',
 'sites',
 'still',
 'their',
 'infancy',
 'Various',
 'versions',
 'have',
 'evolved',
 'over',
 'years',
 'sometimes',
 'accident',
 'sometimes',
 'purpose',
 'injected',
 'humour',
 'like']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [17]:
string = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
prog = re.compile(r'@[\w.]+')
prog.findall(string)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']